In [ ]:
#Created and saves the dataset
#Saves me from loading dataset from folder each training session(takes approximately 4 minutes)

In [1]:
import os
os.add_dll_directory("C:\Program Files\\NVIDIA GPU Computing Toolkit\CUDA\\v10.1\\bin")
os.add_dll_directory("C:\Program Files\\NVIDIA GPU Computing Toolkit\CUDA\\v10.1\libnvvp")
os.add_dll_directory("C:\Program Files\\NVIDIA GPU Computing Toolkit\CUDA\\v10.1")
os.add_dll_directory("C:\Program Files\\NVIDIA GPU Computing Toolkit\CUDA\\v10.1\extras\CUPTI\lib64")
os.add_dll_directory("C:\Program Files\\NVIDIA GPU Computing Toolkit\CUDA\\v10.1\include")
os.add_dll_directory("C:\Program Files\\NVIDIA Corporation\\Nsight Compute 2019.4.0")
import tensorflow as tf
import numpy as np

In [2]:
import sys 
sys.path.insert(0, 'F:\messy code\\afg ordered\second generation')
import data

In [3]:
#returns the paths of images that belong to games published after 2004
def paths(img_dir):
    img_paths = data.dir_to_list(img_dir)
    result = []
    dates = data.dates_directory("F:\messy code\\afg ordered\second generation\image_collecting\getchu\data.txt")
    for path in img_paths: 
        name = data.path_to_name(path)
        if int(dates[data.name_to_id(name)]) < 2005:
            continue
        result.append(path )
    return np.asarray(result)

In [13]:
paths = paths("F:\messy code\\afg ordered\second generation\\faces")

In [14]:
dataset = tf.data.Dataset.from_tensor_slices(paths)

In [15]:
next(iter(dataset))

<tf.Tensor: shape=(), dtype=string, numpy=b'F:\\messy code\\afg ordered\\second generation\\faces\\118x118_to_127x127\\725092char1_face0.jpg'>

In [11]:
feature_dir = "F:\messy code\\afg ordered\second generation\getchu hopefully last\content\\features"

In [10]:
def name_to_picture(path_tensor):
    path = path_tensor.numpy().decode("utf-8")
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [128, 128])

    name = data.path_to_name(path)
    feature_path = data.name_to_feature_path(name , feature_dir)

    feature_vector = np.load(feature_path)

    feature_len = feature_vector.shape[0]

    feature_vector = feature_vector.reshape(1, feature_len)
    return tf.reshape(image, (1,128,128,3)) , feature_vector


dataset = dataset.map(lambda x: tf.py_function(name_to_picture, [x], Tout = [tf.float32, tf.float32]))

NameError: name &#39;dataset&#39; is not defined

In [17]:
next(iter(dataset))

(<tf.Tensor: shape=(1, 128, 128, 3), dtype=float32, numpy=
 array([[[[1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          ...,
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ]],
 
         [[1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          ...,
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ]],
 
         [[1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          ...,
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ]],
 
         ...,
 
       

In [18]:
tf.data.DatasetSpec.from_value(dataset)

DatasetSpec((TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), TensorSpec(shape=<unknown>, dtype=tf.float32, name=None)), TensorShape([]))

In [19]:
#saves the dataset to be used on training
tf.data.experimental.save(dataset ,'F:\messy code\\afg ordered\second generation\\tf_dataset_2' , compression = 'GZIP')

In [37]:
save_pictures_dir = "F:\messy code\\afg ordered\second generation\\faces_to_use_on_saving"
names = os.listdir(save_pictures_dir)
print(names[0].split(".")[0])


1102485char1_face0


In [38]:
save_dataset = tf.data.Dataset.from_tensor_slices(names)

In [39]:
def name_to_picture_save(path_tensor):
    name = path_tensor.numpy().decode("utf-8")
    path = save_pictures_dir + "\\" + name
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [128, 128])
    feature_path = data.name_to_feature_path( name.split(".")[0], feature_dir)
    

    feature_vector = np.load(feature_path)

    feature_len = feature_vector.shape[0]

    feature_vector = feature_vector.reshape(1, feature_len)

    noise = noise = tf.random.uniform([5, 128])
    return name ,tf.reshape(image, (1,128,128,3)) , feature_vector, noise


save_dataset = save_dataset.map(lambda x: tf.py_function(name_to_picture_save, [x], Tout = [tf.string , tf.float32, tf.float32, tf.float32]))

In [43]:
tf.data.DatasetSpec.from_value(save_dataset)

DatasetSpec((TensorSpec(shape=&lt;unknown&gt;, dtype=tf.string, name=None), TensorSpec(shape=&lt;unknown&gt;, dtype=tf.float32, name=None), TensorSpec(shape=&lt;unknown&gt;, dtype=tf.float32, name=None), TensorSpec(shape=&lt;unknown&gt;, dtype=tf.float32, name=None)), TensorShape([]))

In [42]:
#saves the dataet to be used on saving callback
tf.data.experimental.save(save_dataset ,'F:\messy code\\afg ordered\second generation\\save_dataset' , compression = 'GZIP')